<a href="https://colab.research.google.com/github/diegoOrd123/Examen-Primer-parcial/blob/main/Primer_parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las librerias necesarias

In [3]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [5]:
input_layer_size  = 14
num_labels = 5
data = pd.read_csv('/content/drive/MyDrive/Dataset/Base.csv', delimiter=';')
print(data.shape)

data.info()

(1000000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   income                        1000000 non-null  float64
 1   name_email_similarity         1000000 non-null  float64
 2   prev_address_months_count     1000000 non-null  int64  
 3   current_address_months_count  1000000 non-null  int64  
 4   customer_age                  1000000 non-null  int64  
 5   intended_balcon_amount        1000000 non-null  float64
 6   zip_count_4w                  1000000 non-null  int64  
 7   velocity_6h                   1000000 non-null  float64
 8   velocity_24h                  1000000 non-null  float64
 9   velocity_4w                   1000000 non-null  float64
 10  credit_risk_score             1000000 non-null  int64  
 11  bank_months_count             1000000 non-null  int64  
 12  proposed_credit

In [18]:
def generate_synthetic_data(num_samples):
    data = {
        'income': np.random.uniform(0.1, 0.9, num_samples),
        'name_email_similarity': np.random.uniform(0, 1, num_samples),
        'prev_address_months_count': np.random.randint(-1, 383, num_samples),
        'current_address_months_count': np.random.randint(19, 428, num_samples),
        'customer_age': np.random.randint(10, 90, num_samples),
        'intended_balcon_amount': np.random.uniform(-15.5, 113, num_samples),
        'zip_count_4w': np.random.randint(1, 6700, num_samples),
        'velocity_6h': np.random.uniform(3440, 16700, num_samples),
        'velocity_24h': np.random.uniform(1300, 9500, num_samples),
        'velocity_4w': np.random.uniform(2830, 6990, num_samples),
        'credit_risk_score': np.random.uniform(-170, 389, num_samples),
        'bank_months_count': np.random.randint(-1, 32, num_samples),
        'proposed_credit_limit': np.random.uniform(190, 2100, num_samples),
        'payment_type': np.random.choice(['AB', 'AA', 'AC', 'AE', 'AD'], num_samples)
    }
    return pd.DataFrame(data)

# Generar el DataFrame sintético
syntheticData = generate_synthetic_data(1000000)
combinedData = pd.concat([data, syntheticData], ignore_index=True)
print(combinedData.shape)

(2000000, 14)


In [21]:
y = combinedData.iloc[:, -1]
print(y)
mapping = {'AB': 0, 'AA': 1, 'AC': 2, 'AE': 3, 'AD': 4}
def convert_column_to_numeric(data, mapping):
    vectorized_map = np.vectorize(mapping.get)
    numeric_data = vectorized_map(data)
    return numeric_data
numeric_data = convert_column_to_numeric(y, mapping)
y = numeric_data
print(y)

0          AA
1          AD
2          AB
3          AB
4          AA
           ..
1999995    AE
1999996    AD
1999997    AE
1999998    AB
1999999    AE
Name: payment_type, Length: 2000000, dtype: object
[1 4 0 ... 3 0 3]


In [22]:
X=combinedData.iloc[:,:-1]
print(X.shape)
def income_group(income):
    if income < 3:
        return 'Low'
    elif 3 <= income < 7:
        return 'Medium'
    else:
        return 'High'
X['income_level'] = X['income'].apply(income_group)
print(X.shape)
print(X)

(2000000, 13)
(2000000, 14)
               income  name_email_similarity  prev_address_months_count  \
0        3.000000e+00           9.865060e+14                         -1   
1        8.000000e+00           6.174260e+15                         -1   
2        8.000000e+00           9.967070e+15                          9   
3        6.000000e+15           4.751000e+15                         11   
4        9.000000e+00           8.423070e+15                         -1   
...               ...                    ...                        ...   
1999995  3.041642e-01           5.783508e-01                        314   
1999996  8.431517e-01           8.542243e-01                        342   
1999997  5.854508e-01           2.380214e-01                        109   
1999998  4.691686e-01           8.829129e-02                        107   
1999999  7.925825e-01           4.561989e-01                        171   

         current_address_months_count  customer_age  intended_balcon_am

In [23]:
X['has_long_stable_address'] = (
    (X['prev_address_months_count'] >= 52) |
    (X['current_address_months_count'] >= 52)
).astype(int)
print(X.shape)
print(X)

(2000000, 15)
               income  name_email_similarity  prev_address_months_count  \
0        3.000000e+00           9.865060e+14                         -1   
1        8.000000e+00           6.174260e+15                         -1   
2        8.000000e+00           9.967070e+15                          9   
3        6.000000e+15           4.751000e+15                         11   
4        9.000000e+00           8.423070e+15                         -1   
...               ...                    ...                        ...   
1999995  3.041642e-01           5.783508e-01                        314   
1999996  8.431517e-01           8.542243e-01                        342   
1999997  5.854508e-01           2.380214e-01                        109   
1999998  4.691686e-01           8.829129e-02                        107   
1999999  7.925825e-01           4.561989e-01                        171   

         current_address_months_count  customer_age  intended_balcon_amount  \
0     

In [24]:
def risk_group(credit_risk_score):
    if credit_risk_score <= 83:
        return 'Low'
    elif 83 < credit_risk_score <= 122:
        return 'Medium'
    elif 122 < credit_risk_score <= 178:
        return 'High'
    else:
        return 'Very High'
X['risk_category'] = X['credit_risk_score'].apply(risk_group)
print(X.shape)
print(X)

(2000000, 16)
               income  name_email_similarity  prev_address_months_count  \
0        3.000000e+00           9.865060e+14                         -1   
1        8.000000e+00           6.174260e+15                         -1   
2        8.000000e+00           9.967070e+15                          9   
3        6.000000e+15           4.751000e+15                         11   
4        9.000000e+00           8.423070e+15                         -1   
...               ...                    ...                        ...   
1999995  3.041642e-01           5.783508e-01                        314   
1999996  8.431517e-01           8.542243e-01                        342   
1999997  5.854508e-01           2.380214e-01                        109   
1999998  4.691686e-01           8.829129e-02                        107   
1999999  7.925825e-01           4.561989e-01                        171   

         current_address_months_count  customer_age  intended_balcon_amount  \
0     